In [1]:
import torch



In [4]:
#[class1-3, student, scores]

a = torch.rand(3, 32, 8)
b = torch.rand(6, 32, 8)
c = torch.rand(3, 32, 8)

print(torch.cat([a, b], dim=0).shape)

print(torch.stack([a,c], dim =0).shape)

torch.Size([9, 32, 8])
torch.Size([2, 3, 32, 8])


In [10]:
a = torch.rand(5, 32, 8)
b = torch.split(a, 2, 0)
print(b[0].shape)
print(b[1].shape)
print(b[2].shape)
print(len(b))


c = torch.chunk(a, 2, 0)
print(c[0].shape)
print(c[1].shape)
print(len(c))

torch.Size([2, 32, 8])
torch.Size([2, 32, 8])
torch.Size([1, 32, 8])
3
torch.Size([3, 32, 8])
torch.Size([2, 32, 8])
2


In [15]:
a = torch.tensor(3.1415926)

# floor
print(a.floor())

# ceil
print(a.ceil())

# round
print(a.round())

# trunc
print(a.trunc())

# frac
print(a.frac())

tensor(3.)
tensor(4.)
tensor(3.)
tensor(3.)
tensor(0.1416)


In [17]:
a = torch.tensor([1.,2.,3.,4.,5.,6.,7.])
print(a)

print(a.mean())
print(a.max())
print(a.min())
print(a.sum())
print(a.prod())

# argmax  /    argmin
print(a.argmax())
print(a.argmin())

tensor([1., 2., 3., 4., 5., 6., 7.])
tensor(4.)
tensor(7.)
tensor(1.)
tensor(28.)
tensor(5040.)
tensor(6)
tensor(0)


In [20]:
a = torch.ones(3,3)
b = torch.eye(3,3)
print(a)
print(b)

print(torch.eq(a,b))
print(torch.equal(a, b))

tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])
tensor([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]])
tensor([[ True, False, False],
        [False,  True, False],
        [False, False,  True]])
False


In [21]:
# 基于pytorch 实现 手写数字的识别问题 mnist

In [23]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim

In [29]:
if torch.cuda.is_available():
    device = 'cuda'
else: 
    device = 'cpu'

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        # 28*28 = 784
        self.fc1 = nn.Linear(784, 100)
        self.fc2 = nn.Linear(100, 10)
    # hook
    def forward(self, x):
        x = torch.flatten(x, start_dim = 1)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        
        return x
    
    
max_epochs = 5
batch_size = 16

# data
transform = transforms.Compose([transforms.ToTensor()])
# 55000
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)
testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False)

# net init

net = Net()
net.to(device)

# nn. MSE
loss = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr = 0.0001)

def train():
    acc_num=0
    for epoch in range(max_epochs):
        for i,(data, label) in enumerate(train_loader):
            data = data.to(device)
            label = label.to(device)
            optimizer.zero_grad()
            output = net(data)
            Loss = loss(output, label)
            Loss.backward()
            optimizer.step()
            
            pred_class = torch.max(output, dim=1)[1]
            acc_num += torch.eq(pred_class, label.to(device)).sum().item()
            train_acc = acc_num / len(trainset)
    
        net.eval()
        acc_num = 0.0
        best_acc =0
        with torch.no_grad():
            for val_data in test_loader:
                val_image, val_label = val_data
                output = net(val_image.to(device))
                predict_y = torch.max(output, dim=1)[1]
                acc_num += torch.eq(predict_y, val_label.to(device)).sum().item()
            val_acc = acc_num / len(testset)
            print(train_acc, val_acc)
            if val_acc > best_acc:
                torch.save(net.state_dict(), './minst.pth')
                best_acc = val_acc
            
            acc_num = 0
            train_acc = 0
            test_acc = 0
    print('done')

train()

0.22868333333333332 0.3601
0.47163333333333335 0.5569
0.5959666666666666 0.6348
0.6523833333333333 0.6823
0.68475 0.7109
done
